In [151]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import math
warnings.filterwarnings("ignore")

In [152]:
data=pd.read_csv("data\Transformed_data.csv")

In [153]:
data.head()


,ID,Age,Income,Student,Credit-rating,buys computer
0,1,young,high,no,fair,no
1,2,young,high,no,good,no
2,3,middle,high,no,fair,yes
3,4,old,medium,no,fair,yes
4,5,old,low,yes,fair,yes


In [154]:
data.columns=[column.replace(" ","_").lower() for column in data.columns]
data.head()

,id,age,income,student,credit-rating,buys_computer
0,1,young,high,no,fair,no
1,2,young,high,no,good,no
2,3,middle,high,no,fair,yes
3,4,old,medium,no,fair,yes
4,5,old,low,yes,fair,yes


In [155]:
data

,id,age,income,student,credit-rating,buys_computer
0,1,young,high,no,fair,no
1,2,young,high,no,good,no
2,3,middle,high,no,fair,yes
3,4,old,medium,no,fair,yes
4,5,old,low,yes,fair,yes
5,6,old,low,yes,good,no
6,7,middle,low,yes,good,yes
7,8,young,medium,no,fair,no
8,9,young,low,yes,fair,yes
9,10,old,medium,yes,fair,yes


In [156]:
# Calculating entropy of the target attribute
def entropy(data,col):
    uniq_vals={}
    for i in data[col]:
        if i in uniq_vals:
            uniq_vals[i]+=1
        else:
            uniq_vals[i]=1
    n=len(data)
    entropy=0
   
    for i in uniq_vals:
        prob=uniq_vals[i]/n
        entropy-=prob*math.log2(prob)
    
    return entropy
        

    
    

In [157]:
# calculating entropy for each attribute
data.columns

Index(['id', 'age', 'income', 'student', 'credit-rating', 'buys_computer'], dtype='object')

In [158]:
exclude=['id','buys_computer']
cols=data.columns.to_list()
for i in exclude:
    cols.remove(i)
cols

['age', 'income', 'student', 'credit-rating']

In [159]:
def weighted_entropy(data,col,target):
    weighted_entropy=0
    n=len(data)
    uniq_vals={}
    for i in data[col]:
        if i in uniq_vals:
            uniq_vals[i]+=1
        else:
            uniq_vals[i]=1
   
    for i in uniq_vals:
        subset=data[data[col]==i]
        weighted_entropy+=uniq_vals[i]/n*entropy(subset,target)
        
    
    return round(weighted_entropy,3)

entro=[]
for i in cols:
    entro.append(weighted_entropy(data,i,'buys_computer'))
entro
    
#weighted_entropy(data,'income','buys_computer')  

[0.694, 0.911, 0.788, 0.892]

In [160]:
def information_gain(data,col,target):
    entropy_target=entropy(data,target)
    entropy_column=weighted_entropy(data,col,target)
    information_gain=entropy_target-entropy_column
    return round(information_gain,3)

ig=[]
for i in cols:
    ig.append(information_gain(data,i,'buys_computer'))
ig

[0.246, 0.029, 0.152, 0.048]

In [161]:
def splitinfo(data,col):
    n=len(data)
    uniq_vals={}
    split=0
    for i in data[col]:
        if i in uniq_vals:
            uniq_vals[i]+=1
        else:
            uniq_vals[i]=1
    for i in uniq_vals:
       prob=uniq_vals[i]/n
       split-=prob*math.log2(prob)
    return round(split,3)

sp=[]
for i in cols:
    sp.append(splitinfo(data,i))
sp
    

[1.577, 1.557, 1.0, 0.985]

In [162]:
def gainratio(data,col,target):
    gain_col=information_gain(data,col,target)
    split=splitinfo(data,col)
    gain_ratio=gain_col/split
    return round(gain_ratio,3)
gr=[]
for i in cols:
    gr.append(gainratio(data,i,'buys_computer'))
gr

[0.156, 0.019, 0.152, 0.049]

In [163]:
def gini_index(data,col):
    gini=0
    uniq_vals={}
    for i in data[col]:
        if i in uniq_vals:
            uniq_vals[i]+=1
        else:
            uniq_vals[i]=1
    n=len(data)
    for i in uniq_vals:
        prob=uniq_vals[i]/n
        gini+=prob*prob
    gini=1-gini
    return gini
gini_index(data,'buys_computer')

0.4591836734693877

In [164]:
def weighted_gini(data,col,target):
    weighted_gini=0
    n=len(data)
    uniq_vals={}
    for i in data[col]:
        if i in uniq_vals:
            uniq_vals[i]+=1
        else:
            uniq_vals[i]=1
   
    for i in uniq_vals:
        subset=data[data[col]==i]
        weighted_gini+=uniq_vals[i]/n*gini_index(subset,target)
        
    
    return round(weighted_gini,3)

gi=[]
for i in cols:
    gi.append(weighted_gini(data,i,'buys_computer'))
gi

[0.343, 0.44, 0.367, 0.429]

In [165]:
df=pd.DataFrame({'Entropy':entro,'Information_Gain':ig,'Split_Info':sp,'Gain_Ratio':gr,'Gini_Index':gi})
df.index=cols
df

,Entropy,Information_Gain,Split_Info,Gain_Ratio,Gini_Index
age,0.694,0.246,1.577,0.156,0.343
income,0.911,0.029,1.557,0.019,0.440
student,0.788,0.152,1.000,0.152,0.367
credit-rating,0.892,0.048,0.985,0.049,0.429
